In [2]:
from torch import nn
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm
class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

class TCN(nn.Module):
    def __init__(self, input_size, output_size, num_channels, kernel_size, dropout):
        super(TCN, self).__init__()
        self.tcn = TemporalConvNet(input_size, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.linear = nn.Linear(num_channels[-1], output_size)
        self.init_weights()

    def init_weights(self):
        self.linear.weight.data.normal_(0, 0.01)

    def forward(self, x):
        y1 = self.tcn(x)
        return self.linear(y1[:, :, -1])
import torch
import argparse
import torch.optim as optim
import torch.nn.functional as F
import sys
import torch
import numpy as np
from torch.autograd import Variable


def data_generator(N, seq_length):
    """
    Args:
        seq_length: Length of the adding problem data
        N: # of data in the set
    """
    X_num = torch.rand([N, 1, seq_length])
    X_mask = torch.zeros([N, 1, seq_length])
    Y = torch.zeros([N, 1])
    for i in range(N):
        positions = np.random.choice(seq_length, size=2, replace=False)
        X_mask[i, 0, positions[0]] = 1
        X_mask[i, 0, positions[1]] = 1
        Y[i,0] = X_num[i, 0, positions[0]] + X_num[i, 0, positions[1]]
    X = torch.cat((X_num, X_mask), dim=1)
    return Variable(X), Variable(Y)




In [12]:
batch_size=32
cuda=False
dropout=0.0
#                     help='dropout applied to layers (default: 0.0)')
clip=-1.0
#                     help='gradient clip, -1 means no clip (default: -1)')
epochs=10
ksize=7
#                     help='kernel size (default: 7)')
levels=8
#                     help='# of levels (default: 8)')
seq_len=400
#                     help='sequence length (default: 400)')
log_interval=100
#                     help='report interval (default: 100')
lr=4e-3
#                     help='initial learning rate (default: 4e-3)')

#                     help='optimizer to use (default: Adam)')
nhid=30
#                     help='number of hidden units per layer (default: 30)')
seed=1111
#                     help='random seed (default: 1111)')

In [13]:
torch.manual_seed(seed)

input_channels = 2
n_classes = 1
batch_size = batch_size
seq_length = seq_len
epochs = epochs


print("Producing data...")
X_train, Y_train = data_generator(50000, int(seq_length))
X_test, Y_test = data_generator(1000, int(seq_length))


# Note: We use a very simple setting here (assuming all levels have the same # of channels.
channel_sizes = [nhid]*levels
kernel_size = ksize
dropout = dropout
model = TCN(input_channels, n_classes, channel_sizes, kernel_size=kernel_size, dropout=dropout)

if cuda:
    model.cuda()
    X_train = X_train.cuda()
    Y_train = Y_train.cuda()
    X_test = X_test.cuda()
    Y_test = Y_test.cuda()

lr = lr
optimizer = torch.optim.Adam(model.parameters(), lr=lr)


def train(epoch):
    global lr
    model.train()
    batch_idx = 1
    total_loss = 0
    for i in range(0, X_train.size(0), batch_size):
        if i + batch_size > X_train.size(0):
            x, y = X_train[i:], Y_train[i:]
        else:
            x, y = X_train[i:(i+batch_size)], Y_train[i:(i+batch_size)]
        optimizer.zero_grad()
        output = model(x)
        loss = F.mse_loss(output, y)
        loss.backward()
        if clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        batch_idx += 1
        total_loss += loss.item()

        if batch_idx % log_interval == 0:
            cur_loss = total_loss / log_interval
            processed = min(i+batch_size, X_train.size(0))
            print('Train Epoch: {:2d} [{:6d}/{:6d} ({:.0f}%)]\tLearning rate: {:.4f}\tLoss: {:.6f}'.format(
                epoch, processed, X_train.size(0), 100.*processed/X_train.size(0), lr, cur_loss))
            total_loss = 0


def evaluate():
    model.eval()
    with torch.no_grad():
        output = model(X_test)
        test_loss = F.mse_loss(output, Y_test)
        print('\nTest set: Average loss: {:.6f}\n'.format(test_loss.item()))
        return test_loss.item()


for ep in range(1, epochs+1):
    train(ep)
    tloss = evaluate()

Producing data...
Train Epoch:  1 [  3168/ 50000 (6%)]	Learning rate: 0.0040	Loss: 0.324636
Train Epoch:  1 [  6368/ 50000 (13%)]	Learning rate: 0.0040	Loss: 0.173538
Train Epoch:  1 [  9568/ 50000 (19%)]	Learning rate: 0.0040	Loss: 0.176563
Train Epoch:  1 [ 12768/ 50000 (26%)]	Learning rate: 0.0040	Loss: 0.176772
Train Epoch:  1 [ 15968/ 50000 (32%)]	Learning rate: 0.0040	Loss: 0.172667
Train Epoch:  1 [ 19168/ 50000 (38%)]	Learning rate: 0.0040	Loss: 0.170086
Train Epoch:  1 [ 22368/ 50000 (45%)]	Learning rate: 0.0040	Loss: 0.174982
Train Epoch:  1 [ 25568/ 50000 (51%)]	Learning rate: 0.0040	Loss: 0.168434
Train Epoch:  1 [ 28768/ 50000 (58%)]	Learning rate: 0.0040	Loss: 0.166285
Train Epoch:  1 [ 31968/ 50000 (64%)]	Learning rate: 0.0040	Loss: 0.173971
Train Epoch:  1 [ 35168/ 50000 (70%)]	Learning rate: 0.0040	Loss: 0.168752
Train Epoch:  1 [ 38368/ 50000 (77%)]	Learning rate: 0.0040	Loss: 0.167044
Train Epoch:  1 [ 41568/ 50000 (83%)]	Learning rate: 0.0040	Loss: 0.166367
Train Ep

In [16]:
X_train.shape

torch.Size([50000, 2, 400])

In [6]:
""" A very simple Temporal Convolutional Networks baseline
* TCN implementation is adapted from https://github.com/locuslab/TCN
* This is a very small dataset, so it's very easy to overfit. Tune carefully.
* Check the "Output" tab for training logs.
"""
from subprocess import call
import os

FNULL = open(os.devnull, 'w')
call("pip install https://github.com/ceshine/pytorch_helper_bot/archive/0.0.3.zip".split(" "), stdout=FNULL, stderr=FNULL)

# set SEED
os.environ["SEED"] = "42"

DEVICE = "cpu"

# ======================
#     TCN Components
# ======================
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, dropout=0.2):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv2d(n_inputs, n_outputs, (1, kernel_size),
                                           stride=stride, padding=0, dilation=dilation))
        self.pad = torch.nn.ZeroPad2d((padding, 0, 0, 0))
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        self.conv2 = weight_norm(nn.Conv2d(n_outputs, n_outputs, (1, kernel_size),
                                           stride=stride, padding=0, dilation=dilation))
        self.net = nn.Sequential(self.pad, self.conv1, self.relu, self.dropout,
                                 self.pad, self.conv2, self.relu, self.dropout)
        self.downsample = nn.Conv1d(
            n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.01)
        self.conv2.weight.data.normal_(0, 0.01)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.01)

    def forward(self, x):
        out = self.net(x.unsqueeze(2)).squeeze(2)
        res = x if self.downsample is None else self.downsample(x)
        return self.relu(out + res)


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=2, dropout=0.2):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)


# ======================
#     Dataset Utils
# ======================
from pathlib import Path

import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset


def get_dataset(x, y):
    return TensorDataset(
        torch.from_numpy(x).float(),
        torch.from_numpy(y).float()
    )


def get_dataloader(x: np.array, y: np.array, batch_size: int, shuffle: bool = True, num_workers: int = 0):
    dataset = get_dataset(x, y)
    return DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers
    )


def get_ndarray(embedding_values):
    results = []
    for row in embedding_values:
        arr = np.array(row)
        results.append(
            np.pad(arr, ((10 - arr.shape[0], 0), (0, 0)), 'constant')
        )
    # shape: (examples, emb_dim, seq_length)
    return np.transpose(np.stack(results), (0, 2, 1))


def read_dataset(data_dir=Path("data/")):
    if isinstance(data_dir, str):
        data_dir = Path(data_dir)
    df_train = pd.read_json(data_dir / 'train.json')
    df_test = pd.read_json(data_dir / 'test.json')
    x_train = get_ndarray(df_train.audio_embedding)
    y_train = df_train.is_turkey.values
    x_test = get_ndarray(df_test.audio_embedding)
    test_id = df_test.vid_id
    return x_train, y_train, x_test, test_id


# ===============================================
#     Model Creation, Training, and Inference
# ==============================================
import logging

from helperbot.bot import BaseBot
from helperbot.lr_scheduler import TriangularLR
from helperbot.weight_decay import WeightDecayOptimizerWrapper
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss, roc_auc_score
import torch.nn as nn
import pandas as pd

class TurkeyBot(BaseBot):
    name = "Turkey"

    def __init__(self, model, train_loader, val_loader, *, optimizer,
                 avg_window=20, log_dir="./cache/logs/",
                 log_level=logging.INFO, checkpoint_dir="./cache/model_cache/"):
        super().__init__(
            model, train_loader, val_loader,
            optimizer=optimizer, avg_window=avg_window,
            log_dir=log_dir, log_level=log_level, checkpoint_dir=checkpoint_dir,
            batch_idx=0, echo=False, device=DEVICE
        )
        self.criterion = torch.nn.BCEWithLogitsLoss()
        self.loss_format = "%.8f"


class TCNModel(nn.Module):
    def __init__(self, num_channels, kernel_size=2, dropout=0.2):
        super(TCNModel, self).__init__()
        self.tcn = TemporalConvNet(
            128, num_channels, kernel_size=kernel_size, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.decoder = nn.Linear(num_channels[-1], 1)

    def forward(self, x):
        return self.decoder(self.dropout(self.tcn(x)[:, :, -1]))
        
        
def main():
    x_train, y_train, x_test, test_id = read_dataset("../input/")

    test_loader = get_dataloader(
        x_test, np.zeros(x_test.shape[0]), batch_size=128, shuffle=False)

    test_pred_list, val_losses = [], []
    kf = StratifiedKFold(n_splits=8, random_state=31829, shuffle=True)
    for train_index, valid_index in kf.split(x_train, y_train):
        train_loader = get_dataloader(
            x_train[train_index], y_train[train_index],
            batch_size=32, shuffle=True
        )
        val_loader = get_dataloader(
            x_train[valid_index], y_train[valid_index],
            batch_size=128, shuffle=False
        )

        model = TCNModel(num_channels=[20] * 2, kernel_size=3, dropout=0.25)
        model.to(DEVICE)
        optimizer = torch.optim.Adam(
            model.parameters(), betas=(0.9, 0.999), lr=1e-3, weight_decay=0)
        # optimizer = WeightDecayOptimizerWrapper(
        #     optimizer, weight_decay=5e-3
        # )
        batches_per_epoch = len(train_loader)
        bot = TurkeyBot(
            model, train_loader, val_loader,
            optimizer=optimizer, avg_window=batches_per_epoch
        )
        n_steps = batches_per_epoch * 20
        scheduler = TriangularLR(
            optimizer, max_mul=8, ratio=9,
            steps_per_cycle=n_steps
        )
        bot.train(
            n_steps,
            log_interval=batches_per_epoch // 2,
            snapshot_interval=batches_per_epoch,
            early_stopping_cnt=10, scheduler=scheduler)
        val_preds = torch.sigmoid(bot.predict_avg(
            val_loader, k=3, is_test=True).cpu()).numpy().clip(1e-5, 1-1e-5)
        loss = log_loss(y_train[valid_index], val_preds)
        if loss > 0.2:
            # Ditch folds that perform terribly
            bot.remove_checkpoints(keep=0)
            continue
        print("AUC: %.6f" % roc_auc_score(y_train[valid_index], val_preds))
        print("Val loss: %.6f" % loss)
        val_losses.append(loss)
        test_pred_list.append(torch.sigmoid(bot.predict_avg(
            test_loader, k=3, is_test=True).cpu()).numpy().clip(1e-5, 1-1e-5))
        bot.remove_checkpoints(keep=0)

    val_loss = np.mean(val_losses)
    test_preds = np.mean(test_pred_list, axis=0)
    print("Validation losses: %.6f +- %.6f" %
          (np.mean(val_losses), np.std(val_losses)))

    df_sub = pd.DataFrame({
        "vid_id": test_id,
        "is_turkey": test_preds
    })
    df_sub.to_csv("submission.csv", index=False)

main()

ValueError: ignored